In [1]:
import tensorflow as tf
import numpy as np
from  keras.utils import np_utils
from matplotlib import pyplot as plt
import pandas as pd
import os 

In [2]:
print(tf.__version__)

2.3.0


In [3]:
def generate_train_header(x, y, file='tain_data.h'):
    '''
    this method generate the
    :param x:  input x data size
    :param y:  input label (one hot label)
    :return:
    '''
    # quantize input x
    min_value = np.min(x)
    max_value = np.max(x)

    int_bits = int(np.ceil(np.log2(max(abs(min_value), abs(max_value)))))
    dec_bits = 7 - int_bits
    x = np.round(x*2**dec_bits).astype(np.int8)
    
#     print(x[0][0])
    data = x.astype(dtype="byte")
    label = y
    node = 0
    with open(file, 'w') as f:
        num_of_image = x.shape[0]
        
        print("Number of training images:",num_of_image)
        for i in range(num_of_image):
            f.write('#define NODE_%d_TRAIN_IMG_%d {'%(node,i) )
            (data[i]).flatten().tofile(f, sep=", ") # convert 0~1 to 0~127
            f.write('} \n')
            f.write('#define NODE_%d_TRAIN_IMG%d_LABEL'% (node, i))
            f.write(' %d \n \n' % label[i])
        f.write('#define NODE_%d_TOTAL_TRAIN_IMAGES %d \n \n'%(node,num_of_image))

        f.write('static q7_t NODE_%d_TRAIN_IMAGES[%d][%d] = {' % (node,num_of_image, data[0].flatten().shape[0]))
        f.write('NODE_%d_TRAIN_IMG_0'%(node))
        for i in range(num_of_image -1):
            f.write(',NODE_%d_TRAIN_IMG_%d'%(node, i+1))
        f.write('};\n\n')

        f.write('static q7_t NODE_%d_TRAIN_LABELS[%d] = {' % (node,num_of_image))
        f.write('NODE_%d_TRAIN_IMG0_LABEL'%(node))
        for i in range(num_of_image -1):
            f.write(',NODE_%d_TRAIN_IMG%d_LABEL'%(node, i+1))
        f.write('};\n\n')

#     return 000


In [4]:
def generate_test_bin(x, y, file='test_data.h'):
    '''
    method to generate the test data as a header file
    :param x:  input x data size
    :param y:  input label 
    :write to file 
    :return: nothing
    '''
    # quantize input x
    min_value = np.min(x)
    max_value = np.max(x)

    int_bits = int(np.ceil(np.log2(max(abs(min_value), abs(max_value)))))
    dec_bits = 7 - int_bits
    x = np.round(x*2**dec_bits).astype(np.int8)
    
    data = x
    label = y

    with open(file, 'w') as f:
        num_of_image = x.shape[0]
        print("Number of test images:",num_of_image)
        for i in range(num_of_image):
            f.write('#define TEST_IMG%d {'% (i))
            data[i].flatten().tofile(f, sep=", ")
            f.write('} \n')
            f.write('#define TEST_IMG%d_LABEL'% (i))
            f.write(' %d \n \n' % label[i])
        f.write('#define TOTAL_TEST_IMAGES %d \n \n'%(num_of_image))

        f.write('static q7_t TEST_IMAGES[%d][%d] = {' % (num_of_image, data[0].flatten().shape[0]))
        f.write('TEST_IMG0')
        for i in range(num_of_image -1):
            f.write(',TEST_IMG%d'%(i+1))
        f.write('};\n\n')

        f.write('static q7_t TEST_LABELS[%d] = {' % (num_of_image))
        f.write('TEST_IMG0_LABEL')
        for i in range(num_of_image -1):
            f.write(',TEST_IMG%d_LABEL'%(i+1))
        f.write('};\n\n')


## HAR

In [5]:
import pandas as pd
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler

In [6]:

columns = ['user','activity','timestamp', 'x-axis', 'y-axis', 'z-axis']
df = pd.read_csv('WISDM_ar_v1.1_raw.txt', header = None, names = columns, comment=';')
df = df.dropna()

N_TIME_STEPS = 90
N_FEATURES = 3
step = 20
segments = []
labels = []
for i in range(0, len(df) - N_TIME_STEPS, step):
    xs = df['x-axis'].values[i: i + N_TIME_STEPS]
    ys = df['y-axis'].values[i: i + N_TIME_STEPS]
    zs = df['z-axis'].values[i: i + N_TIME_STEPS]
    label = stats.mode(df['activity'][i: i + N_TIME_STEPS])[0][0]
    segments.append([xs, ys, zs])
    
    labels.append(label)


# scaler = MinMaxScaler()


reshaped_segments = np.asarray(segments,dtype = np.float32).reshape(-1, N_TIME_STEPS, N_FEATURES)


scalers = {}
for i in range(reshaped_segments.shape[1]):
    scalers[i] = StandardScaler()
    reshaped_segments[:, i, :] = scalers[i].fit_transform(reshaped_segments[:, i, :]) 


labels = LabelEncoder().fit_transform(labels)


In [7]:
x_train, x_test, y_train, y_test = train_test_split(reshaped_segments, labels, test_size=1-0.25 )

In [8]:
y_test

array([5, 1, 3, ..., 5, 4, 1])

In [ ]:

for i in range (30):
    labelindices = np.random.choice(6, 3, replace=False)
    print(labelindices)

    # generate_train_header(x_train[:1500], y_train[:1500], file='cifar_train_data.h')
    # generate_test_bin(x_test[:1200], y_test[:1200], file='cifar_test_data.h')


    subset_x_train = x_train[np.isin(y_train, labelindices).flatten()]
    subset_y_train = y_train[np.isin(y_train, labelindices).flatten()]
    # subset_y_train, _ = pd.factorize(np.concatenate( subset_y_train, axis=0 ))  

    subset_y_train = np.unique(subset_y_train, return_inverse=True)[1] 


    subset_x_test = x_test[np.isin(y_test, labelindices).flatten()]
    subset_y_test = y_test[np.isin(y_test, labelindices).flatten()] 


    subset_y_test = np.unique(subset_y_test, return_inverse=True)[1] 
    # generate_test_bin(subset_x_test, subset_y_test, file='cifar_test_cmsis.h')


    # subset_y_test, _ = pd.factorize(np.concatenate( subset_y_test, axis=0 ))  


    # subset_x_train = x_train[np.isin(y_train, [7,8,9]).flatten()] 
    # subset_y_train = y_train[np.isin(y_train, [7,8,9]).flatten()]- 7

    # subset_x_test = x_test[np.isin(y_test, [7,8,9]).flatten()]
    # subset_y_test = y_test[np.isin(y_test, [7,8,9]).flatten()]- 7

    indices = np.random.choice(subset_x_train.shape[0], 500, replace=False)
#     print(subset_x_train.shape[0])
    subset_x_train = subset_x_train[indices]
    subset_y_train= subset_y_train[indices]
    

    generate_train_header(subset_x_train, subset_y_train, file='./src/har_train_data.h')
    generate_test_bin(subset_x_test, subset_y_test, file='./src/har_test_data.h')


    # generate_train_header(x_train ,y_train, file='cifar_train_data.h')
    # generate_test_bin(x_test, y_test, file='cifar_test_data.h')


    # generate_train_header(subset_x_train, subset_y_train, file='cifar_train_data.h')
    # subset_x_train.shape[0]
    # federated_train_images(0, subset_x_train*127, subset_y_train, subset_x_train.shape[0], file='CIFAR3-TrainSet.h')

    # subset_x_test.shape[0]

    # imagesTestcfile(subset_x_test*127, subset_y_test,subset_x_test.shape[0] , file='CIFAR3-TestSet.h')


    print ("done")
    !make 
    !./MiniLearn

[2 3 4]
Number of training images: 500
Number of test images: 8710
done
gcc -Iinclude/ -Isrc/  -I../CMSIS/CMSIS/DSP/Include/ -I../CMSIS/CMSIS/Core/Include -I../CMSIS/CMSIS/NN/Include  -Wall -Wextra -Wno-unknown-pragmas -Wshift-overflow -ggdb3 -fPIC -g -O2 -g -c ./src/main.c -o obj/main.o
./src/main.c:504:14: warning: unused variable 'blockSize' [-Wunused-variable]
    uint32_t blockSize = sizeof(y_out);
             ^
1 warning generated.
gcc -Iinclude/ -Isrc/  -I../CMSIS/CMSIS/DSP/Include/ -I../CMSIS/CMSIS/Core/Include -I../CMSIS/CMSIS/NN/Include  -Wall -Wextra -Wno-unknown-pragmas -Wshift-overflow -ggdb3 -fPIC -g -O2 -g -c ./src/image.c -o obj/image.o
In file included from ./src/image.c:47:
src/stb_image.h:6361:84: warning: unused parameter 'req_comp'
      [-Wunused-parameter]
  ...*s, stbi__gif *g, int *comp, int req_comp, stbi_uc *two_back)
                                      ^
src/stb_image.h:6585:107: warning: unused parameter 'ri' [-Wunused-parameter]
  ...*s, int *x, int *y,

ar rcs libMiniLearn.a obj/main.o obj/image.o obj/args.o obj/matrix.o obj/list.o obj/data.o obj/classifier.o obj/net.o obj/maxpool_layer.o obj/convolutional_layer.o obj/connected_layer.o obj/activations.o obj/arm_fully_connected_q7.o obj/arm_convolve_HWC_q7_basic.o obj/arm_convolve_HWC_q7_fast.o obj/arm_relu_q7.o obj/arm_pool_q7_HWC.o obj/arm_convolve_HWC_q7_RGB.o obj/arm_softmax_q7.o obj/batchnorm_layer.o obj/activation_layer.o
/Library/Developer/CommandLineTools/usr/bin/ranlib: file: libMiniLearn.a(activations.o) has no symbols
gcc -Iinclude/ -Isrc/  -I../CMSIS/CMSIS/DSP/Include/ -I../CMSIS/CMSIS/Core/Include -I../CMSIS/CMSIS/NN/Include  -Wall -Wextra -Wno-unknown-pragmas -Wshift-overflow -ggdb3 -fPIC -g -O2 -g obj/main.o obj/image.o obj/args.o obj/matrix.o obj/list.o obj/data.o obj/classifier.o obj/net.o obj/maxpool_layer.o obj/convolutional_layer.o obj/connected_layer.o obj/activations.o obj/arm_fully_connected_q7.o obj/arm_convolve_HWC_q7_basic.o obj/arm_convolve_HWC_q7_fast.o obj/

1 warning generated.
gcc -Iinclude/ -Isrc/  -I../CMSIS/CMSIS/DSP/Include/ -I../CMSIS/CMSIS/Core/Include -I../CMSIS/CMSIS/NN/Include  -Wall -Wextra -Wno-unknown-pragmas -Wshift-overflow -ggdb3 -fPIC -g -O2 -g -c ./src/classifier.c -o obj/classifier.o
gcc -Iinclude/ -Isrc/  -I../CMSIS/CMSIS/DSP/Include/ -I../CMSIS/CMSIS/Core/Include -I../CMSIS/CMSIS/NN/Include  -Wall -Wextra -Wno-unknown-pragmas -Wshift-overflow -ggdb3 -fPIC -g -O2 -g -c ./src/net.c -o obj/net.o
gcc -Iinclude/ -Isrc/  -I../CMSIS/CMSIS/DSP/Include/ -I../CMSIS/CMSIS/Core/Include -I../CMSIS/CMSIS/NN/Include  -Wall -Wextra -Wno-unknown-pragmas -Wshift-overflow -ggdb3 -fPIC -g -O2 -g -c ./src/maxpool_layer.c -o obj/maxpool_layer.o
gcc -Iinclude/ -Isrc/  -I../CMSIS/CMSIS/DSP/Include/ -I../CMSIS/CMSIS/Core/Include -I../CMSIS/CMSIS/NN/Include  -Wall -Wextra -Wno-unknown-pragmas -Wshift-overflow -ggdb3 -fPIC -g -O2 -g -c ./src/convolutional_layer.c -o obj/convolutional_layer.o
gcc -Iinclude/ -Isrc/  -I../CMSIS/CMSIS/DSP/Include/ 

test acc = 0.088129
1 :: train acc = 0.770000 & test acc = 0.760556
2 :: train acc = 0.782000 & test acc = 0.772196
3 :: train acc = 0.814000 & test acc = 0.800285
4 :: train acc = 0.864000 & test acc = 0.844468
5 :: train acc = 0.898000 & test acc = 0.884019
6 :: train acc = 0.930000 & test acc = 0.914306
7 :: train acc = 0.946000 & test acc = 0.931944
8 :: train acc = 0.952000 & test acc = 0.944237
9 :: train acc = 0.958000 & test acc = 0.951422
10 :: train acc = 0.964000 & test acc = 0.956114
11 :: train acc = 0.972000 & test acc = 0.960805
12 :: train acc = 0.972000 & test acc = 0.964072
13 :: train acc = 0.972000 & test acc = 0.966803
14 :: train acc = 0.978000 & test acc = 0.968882
15 :: train acc = 0.978000 & test acc = 0.971020
16 :: train acc = 0.978000 & test acc = 0.972326
17 :: train acc = 0.978000 & test acc = 0.973514
18 :: train acc = 0.978000 & test acc = 0.975177
19 :: train acc = 0.978000 & test acc = 0.976067
20 :: train acc = 0.978000 & test acc = 0.976661
21 :: tra

4 warnings generated.
gcc -I../CMSIS/CMSIS/DSP/Include/ -I../CMSIS/CMSIS/Core/Include -I../CMSIS/CMSIS/NN/Include  -Wall -Wextra -Wno-unknown-pragmas -Wshift-overflow -ggdb3 -fPIC -g -O2 -g -shared obj/main.o obj/image.o obj/args.o obj/matrix.o obj/list.o obj/data.o obj/classifier.o obj/net.o obj/maxpool_layer.o obj/convolutional_layer.o obj/connected_layer.o obj/activations.o obj/arm_fully_connected_q7.o obj/arm_convolve_HWC_q7_basic.o obj/arm_convolve_HWC_q7_fast.o obj/arm_relu_q7.o obj/arm_pool_q7_HWC.o obj/arm_convolve_HWC_q7_RGB.o obj/arm_softmax_q7.o obj/batchnorm_layer.o obj/activation_layer.o -o libMiniLearn.so -lm -pthread 
ar rcs libMiniLearn.a obj/main.o obj/image.o obj/args.o obj/matrix.o obj/list.o obj/data.o obj/classifier.o obj/net.o obj/maxpool_layer.o obj/convolutional_layer.o obj/connected_layer.o obj/activations.o obj/arm_fully_connected_q7.o obj/arm_convolve_HWC_q7_basic.o obj/arm_convolve_HWC_q7_fast.o obj/arm_relu_q7.o obj/arm_pool_q7_HWC.o obj/arm_convolve_HWC_q7

In [12]:
print ("done")


done


In [ ]:
for i in range (21):
    label3random = np.random.choice(10, 3, replace=False)
    label3random
    subset_x_test = x_test[np.isin(y_test, label3random).flatten()]
    subset_y_test = y_test[np.isin(y_test, label3random).flatten()] 
    # generate_test_bin(subset_x_test, subset_y_test, file='cifar_test_cmsis.h')


    # subset_y_test, _ = pd.factorize(np.concatenate( subset_y_test, axis=0 ))  


    # subset_y_test = np.unique(subset_y_test, return_inverse=True)[1] 



    generate_test_bin(subset_x_test, subset_y_test, file='./src/cifar_test_data.h')
    print("done")
    !make 
    !./uwnet

In [ ]:
# subset_y_test = y_test[np.isin(y_test, labelindices).flatten()] 
# values = np.array([8,2,1,2,8])
# a = np.concatenate( subset_y_test, axis=0 )
# codes, uniques = pd.factorize(a)  # default: na_sentinel=-1

In [ ]:
subset_y_test

In [ ]:
print(subset_x_train.shape)
print(subset_y_test.shape)
print (np.unique(subset_y_test))

In [ ]:
# imgplot = plt.imshow(subset_x_train[21])
# print(subset_y_train[21])